In [151]:
#Setup
import pandas as pd
from db import get_engine

engine = get_engine()

with engine.begin() as connection:
    connection.exec_driver_sql("PRAGMA foreign_keys = ON;")
    connection.exec_driver_sql("ATTACH DATABASE 'raw.db' AS raw;")
    connection.exec_driver_sql("ATTACH DATABASE 'relational.db' AS relational;")

In [152]:
#Validating that all required raw tables exist before performing sanity checks
#Failing indicates that loading raw tables (00_load_raw) was not run
tables_needed = [
    'crm_customers',
    'crm_products',
    'crm_sales',
    'erp_customers',
    'erp_locations',
    'erp_product_categories'
]

tables_loaded = pd.read_sql("""
SELECT name
FROM raw.sqlite_master
WHERE type = 'table'
;""", engine)['name'].tolist()

missing = set(tables_needed) - set(tables_loaded)

if missing:
    print(f'Missing Tables:')
    for m in missing:
        print(f'{m}')
    raise RuntimeError(
    f'Run 00_load_raw.ipynb first')

In [153]:
#printing shape and head of each CSV file
files = {
    'crm_customers': 'datasets/CRM/cust_info.csv',
    'crm_products': 'datasets/CRM/prd_info.csv',
    'crm_sales': 'datasets/CRM/sales_details.csv',
    'erp_customers': 'datasets/ERP/CUST_AZ12.csv',
    'erp_locations': 'datasets/ERP/LOC_A101.csv',
    'erp_product_categories': 'datasets/ERP/PX_CAT_G1V2.csv'
}

for name, location in files.items():
    df = pd.read_csv(location)
    print(f'File: {name}')
    print(f'Shape: {df.shape}')
    display(df.head())

File: crm_customers
Shape: (18494, 7)


,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date
0,11000.0,AW00011000,Jon,Yang,M,M,2025-10-06
1,11001.0,AW00011001,Eugene,Huang,S,M,2025-10-06
2,11002.0,AW00011002,Ruben,Torres,M,M,2025-10-06
3,11003.0,AW00011003,Christy,Zhu,S,F,2025-10-06
4,11004.0,AW00011004,Elizabeth,Johnson,S,F,2025-10-06


File: crm_products
Shape: (397, 7)


,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt
0,210,CO-RF-FR-R92B-58,HL Road Frame - Black- 58,NaN,R,2003-07-01,NaN
1,211,CO-RF-FR-R92R-58,HL Road Frame - Red- 58,NaN,R,2003-07-01,NaN
2,212,AC-HE-HL-U509-R,Sport-100 Helmet- Red,12.0,S,2011-07-01,2007-12-28
3,213,AC-HE-HL-U509-R,Sport-100 Helmet- Red,14.0,S,2012-07-01,2008-12-27
4,214,AC-HE-HL-U509-R,Sport-100 Helmet- Red,13.0,S,2013-07-01,NaN


File: crm_sales
Shape: (60398, 9)


...

,CID,BDATE,GEN
0,NASAW00011000,1971-10-06,Male
1,NASAW00011001,1976-05-10,Male
2,NASAW00011002,1971-02-09,Male
3,NASAW00011003,1973-08-14,Female
4,NASAW00011004,1979-08-05,Female


File: erp_locations
Shape: (18484, 2)


,CID,CNTRY
0,AW-00011000,Australia
1,AW-00011001,Australia
2,AW-00011002,Australia
3,AW-00011003,Australia
4,AW-00011004,Australia


File: erp_product_categories
Shape: (37, 4)


,ID,CAT,SUBCAT,MAINTENANCE
0,AC_BR,Accessories,Bike Racks,Yes
1,AC_BS,Accessories,Bike Stands,No
2,AC_BC,Accessories,Bottles and Cages,No
3,AC_CL,Accessories,Cleaners,Yes
4,AC_FE,Accessories,Fenders,No


### crm_customers

In [131]:
#Checking for NULL values in cst_id
pd.read_sql("""
SELECT *
FROM raw.crm_customers
WHERE cst_id IS NULL
;""", engine)

,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date
0,None,SF566,None,None,None,None,None
1,None,PO25,None,None,None,None,None
2,None,13451235,None,None,None,None,None
3,None,A01Ass,None,None,None,None,None


These columns hold no valuable information

In [132]:
#Looking at rows with repeated cst_id
pd.read_sql("""
SELECT *
FROM raw.crm_customers
WHERE cst_id IN (
    SELECT cst_id
    FROM raw.crm_customers
    GROUP BY cst_id
    HAVING COUNT(*) > 1)
ORDER BY cst_id;
""", engine)

,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date
0,29433.0,AW00029433,None,None,M,M,2026-01-25
1,29433.0,AW00029433,Thomas,King,M,M,2026-01-27
2,29449.0,AW00029449,None,Chen,S,None,2026-01-25
3,29449.0,AW00029449,Laura,Chen,S,F,2026-01-26
4,29466.0,AW00029466,None,None,None,None,2026-01-25
5,29466.0,AW00029466,Lance,Jimenez,M,None,2026-01-26
6,29466.0,AW00029466,Lance,Jimenez,M,M,2026-01-27
7,29473.0,AW00029473,Carmen,None,None,None,2026-01-25
8,29473.0,AW00029473,Carmen,Subram,S,None,2026-01-26
9,29483.0,AW00029483,None,Navarro,None,None,2026-01-25


Cases where cst_id is repeated, the entries with a later return date hold more valuable information than earlier ones

In [133]:
#Checking at non NULL cst_id entries:
#where cst_id, cst_key, cst_firstname, cst_lastname are invalid
pd.read_sql("""
SELECT *
FROM raw.crm_customers
WHERE cst_id IS NOT NULL
AND ((CAST(cst_id AS INTEGER) GLOB '*[^0-9]*'
    OR LENGTH(CAST(cst_id AS INTEGER)) != 5)
    OR cst_key NOT LIKE 'AW00%'
    OR TRIM(cst_firstname) = ''
    OR TRIM(cst_firstname) GLOB '*[^A-Za-zÀ-ÖØ-öø-ÿ]*'
    OR TRIM(cst_lastname) = ''
    OR TRIM(cst_lastname) GLOB '*[^A-Za-zÀ-ÖØ-öø-ÿ]*')
;""", engine)

,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date
0,14974.0,AW00014974,K.,Saravan,S,None,2025-10-12
1,17641.0,AW00017641,Lorrin,Smith-Bates,S,M,2026-01-04
2,20164.0,AW00020164,Carol Ann,Rockne,M,M,2026-01-05
3,20497.0,AW00020497,Francisco,Javier Castrejón,M,F,2026-01-05
4,21190.0,AW00021190,Pablo,Rovira Diez,S,M,2026-01-05
5,22219.0,AW00022219,Juha-Pekka,Posti,M,F,2026-01-05
6,23737.0,AW00023737,Elena,Velez Amezaga,M,F,2026-01-05
7,26299.0,AW00026299,Y.,Yong,S,M,2026-01-14


Name validation above allows standard letters and common accented letters, but flags punctuation and double names, which are still acceptable

In [134]:
#Checking for unique values in cst_marital_status
pd.read_sql("""
SELECT DISTINCT cst_marital_status
FROM raw.crm_customers""", engine)

,cst_marital_status
0,M
1,S
2,None


In [135]:
#Checking for unique values in cst_gndr
pd.read_sql("""
SELECT DISTINCT cst_gndr
FROM raw.crm_customers""", engine)

,cst_gndr
0,M
1,F
2,None


In [136]:
#Checking for invalid (futuristic) create dates
pd.read_sql("""
SELECT *
FROM raw.crm_customers
WHERE cst_create_date IS NOT NULL
AND (DATE(cst_create_date) IS NULL
OR DATE(cst_create_date) > DATE('now'));
""", engine)

,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date
0,27612.0,AW00027612,Lucas,Hill,M,M,2026-01-17
1,27613.0,AW00027613,Alexia,Washington,S,F,2026-01-18
2,27614.0,AW00027614,Warren,Ye,M,M,2026-01-19
3,27615.0,AW00027615,Maria,Henderson,M,F,2026-01-20
4,27616.0,AW00027616,Emma,Brown,S,F,2026-01-21
...,...,...,...,...,...,...,...
1873,29480.0,AW00029480,Nina,Raji,S,None,2026-01-25
1874,29481.0,AW00029481,Ivan,Suri,S,None,2026-01-25
1875,29482.0,AW00029482,Clayton,Zhang,M,None,2026-01-25
1876,29483.0,AW00029483,None,Navarro,None,None,2026-01-25


Some entries have a future cst_create_date therefore these are invalid

In [137]:
#Looking at connections of crm_customers.cst_id to crm.sales.sls_cust_id
pd.read_sql("""
SELECT sls_cust_id
FROM raw.crm_sales
LIMIT 5;
""", engine)

,sls_cust_id
0,21768
1,28389
2,25863
3,14501
4,11003


The standard format for both of these columns are 5 digit values

Based on this exploration, raw.crm_customers will be cleaned up by:
- Removing rows where cst_id, cst_firstname, and cst_lastname is NULL
- Removing cst_id duplicates by keeping the ones with a later create date
- Trimming spaces from text columns and ensuring consistent capitalization
- Editing cst_marital_status to "Married" or "Single"
- Editing cst_gndr to "Male" or "Female"
- Setting cst_create_date as NULL if it is futuristic
- Changing data types
    - cst_id to integer
    - cst_create_date to date

### crm_products

In [138]:
#Checking for NULL values
pd.read_sql("""
SELECT *
FROM raw.crm_products
WHERE prd_id IS NULL
OR prd_key IS NULL
OR prd_nm IS NULL
or prd_cost IS NULL
or prd_cost NOT LIKE '%.0'
;""", engine)

,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt
0,210,CO-RF-FR-R92B-58,HL Road Frame - Black- 58,None,R,2003-07-01,None
1,211,CO-RF-FR-R92R-58,HL Road Frame - Red- 58,None,R,2003-07-01,None


Only two NULL entries in prd_cost, rest of the entries in prd_cost are integers

In [139]:
#Looking at entries with duplicate prd_nm
pd.read_sql("""
SELECT *
FROM raw.crm_products
WHERE prd_nm IN (
    SELECT prd_nm
    FROM raw.crm_products
    GROUP BY prd_nm
    HAVING COUNT(*)>1)
LIMIT 20;""", engine)

,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt
0,212,AC-HE-HL-U509-R,Sport-100 Helmet- Red,12.0,S,2011-07-01,2007-12-28
1,213,AC-HE-HL-U509-R,Sport-100 Helmet- Red,14.0,S,2012-07-01,2008-12-27
2,214,AC-HE-HL-U509-R,Sport-100 Helmet- Red,13.0,S,2013-07-01,None
3,215,AC-HE-HL-U509,Sport-100 Helmet- Black,12.0,S,2011-07-01,2007-12-28
4,216,AC-HE-HL-U509,Sport-100 Helmet- Black,14.0,S,2012-07-01,2008-12-27
5,217,AC-HE-HL-U509,Sport-100 Helmet- Black,13.0,S,2013-07-01,None
6,220,AC-HE-HL-U509-B,Sport-100 Helmet- Blue,12.0,S,2011-07-01,2007-12-28
7,221,AC-HE-HL-U509-B,Sport-100 Helmet- Blue,14.0,S,2012-07-01,2008-12-27
8,222,AC-HE-HL-U509-B,Sport-100 Helmet- Blue,13.0,S,2013-07-01,None
9,223,CL-CA-CA-1098,AWC Logo Cap,6.0,S,2011-07-01,2007-12-28


Seems like the same products were sold for different prices at different times

In [140]:
#Looking at unique prd_line values
pd.read_sql("""
SELECT DISTINCT prd_line
FROM raw.crm_products""", engine)

,prd_line
0,R
1,S
2,M
3,None
4,T


In [141]:
#Counting number of entries where start date is after end date
pd.read_sql("""
SELECT COUNT(*)
FROM raw.crm_products
WHERE DATE(prd_start_dt) > DATE(prd_end_dt)
;""", engine)

,COUNT(*)
0,200


prd_start_dt is often after prd_end_dt

In [142]:
#Looking at connections of crm_products.prd_key to crm_sales.sls_prd_key
pd.read_sql("""
SELECT sls_prd_key
FROM raw.crm_sales
LIMIT 5;""", engine)

,sls_prd_key
0,BK-R93R-62
1,BK-M82S-44
2,BK-M82S-44
3,BK-R50B-62
4,BK-M82S-44


In [143]:
#Looking at connections of crm_products.prd_key to erp_product_categories.ID
pd.read_sql("""
SELECT ID
FROM raw.erp_product_categories
LIMIT 5;""", engine)

,ID
0,AC_BR
1,AC_BS
2,AC_BC
3,AC_CL
4,AC_FE


crm_products.prd_key is a combination of erp_product_categories.ID and crm_sales.sls_prd_key

Based on this exploration, raw.crm_products will be cleaned up by:
- Splitting prd_key into category id and product key
- Changing prd_line to "Mountain", "Road", "Other", "Touring"
- Fixing prd_end_dt to the day before the next start date of the same product
- Trim spaces from text columns
- Changing data types
    - prd_id to integer
    - prd_cost to integer
    - prd_start_dt and prd_end_dt to date
- Splitting the table into two, one for general product information and the other for price history tracking
    - Because crm_sales connects to crm_products through prd_key, which is not unique in crm_products

### crm_sales

In [144]:
#Looking at entries with duplicate sls_ord_num
pd.read_sql("""
SELECT sls_ord_num, sls_prd_key, sls_cust_id
FROM raw.crm_sales
WHERE sls_ord_num IN (
    SELECT sls_ord_num
    FROM raw.crm_sales
    GROUP BY sls_ord_num
    HAVING COUNT (*) > 1)
ORDER BY sls_ord_num
LIMIT 10;""", engine)

,sls_ord_num,sls_prd_key,sls_cust_id
0,SO51176,BK-R89B-58,18239
1,SO51176,BC-R205,18239
2,SO51177,BK-T44U-46,27873
3,SO51177,HL-U509,27873
4,SO51178,BK-M68S-38,11245
5,SO51178,BC-M005,11245
6,SO51178,WB-H098,11245
7,SO51179,BK-R89B-52,22430
8,SO51179,TI-R982,22430
9,SO51179,TT-R982,22430


Orders with multiple products are broken up into multiple entries, one for each product ordered

In [145]:
#Looking at invalid sls_ord_num, sls_order_dt, sls_ship_dt, or sls_due_dt
pd.read_sql("""
SELECT sls_ord_num, sls_order_dt, sls_ship_dt, sls_due_dt
FROM raw.crm_sales
WHERE LENGTH(sls_cust_id) != 5
OR LENGTH(sls_order_dt) != 8
OR LENGTH(sls_ship_dt) != 8
OR LENGTH(sls_due_dt) != 8;""", engine)

,sls_ord_num,sls_order_dt,sls_ship_dt,sls_due_dt
0,SO64338,0,20130823,20130828
1,SO64339,0,20130823,20130828
2,SO64339,0,20130823,20130828
3,SO64340,0,20130823,20130828
4,SO64340,0,20130823,20130828
5,SO64377,0,20130824,20130829
6,SO64377,0,20130824,20130829
7,SO64379,0,20130824,20130829
8,SO64381,0,20130824,20130829
9,SO64623,0,20130827,20130901


Invalid sls_order_dt values are present

In [146]:
#Looking at invalid sls_sales, sls_quantity, or sls_price values
pd.read_sql("""
SELECT sls_sales, sls_quantity, sls_price
FROM raw.crm_sales
WHERE sls_sales <= 0
OR sls_sales IS NULL
OR sls_quantity <= 0
OR sls_quantity IS NULL
OR sls_price <= 0
OR sls_price IS NULL
OR sls_quantity * sls_price != sls_sales
LIMIT 10;""", engine)

,sls_sales,sls_quantity,sls_price
0,10.0,2,NaN
1,25.0,5,NaN
2,70.0,2,NaN
3,9.0,1,NaN
4,35.0,1,NaN
5,100.0,10,NaN
6,16.0,2,NaN
7,769.0,1,-769.0
8,30.0,1,-30.0
9,22.0,1,-22.0


NULL, negative, 0 values present, as well as record where calculated totals do not match with price * quantity

Based on this exploration, raw.crm_sales will be cleaned up by:
- sls_order_dt to NULL if not 8 characters long
- Fixing sls_sales and sls_price
- Changing data types
    - sls_cust_id to integer
    - sls_order_dt, sls_ship_dt, sls_due_dt to date
    - sls_sales, sls_quantity, sls_price to integer

### erp_customers

In [147]:
#Looking at CIDs not following either AW00... or NAS... format
pd.read_sql("""
SELECT CID
FROM raw.erp_customers
WHERE CID NOT LIKE 'AW00%' AND CID NOT LIKE 'NAS%';""", engine)

,CID


CID is formatted as NASAW00... or AW00...

Desired format is AW00... to align with crm_customers.cst_key

In [148]:
#Looking at invalid birth years
pd.read_sql("""
SELECT BDATE
FROM raw.erp_customers
WHERE SUBSTRING(BDATE, 1, 4) > STRFTIME('%Y', 'now')
OR SUBSTRING(BDATE, 1, 4) < '1900'
LIMIT 5;""", engine)

,BDATE
0,2050-07-06
1,2042-02-22
2,2050-05-21
3,2038-10-17
4,2045-03-03


Invalid futuristic birthdays are present

In [149]:
#Looking at unique GEN values
pd.read_sql("""
SELECT DISTINCT GEN
FROM raw.erp_customers;""", engine)

,GEN
0,Male
1,Female
2,
3,
4,M
5,F
6,F
7,M
8,None


GEN column has inconsistent formatting

Based on this exploration, raw.erp_customers will be cleaned up by:
- Remove extra characters from CID so that it matches crm_customers.cst_key
- Replace invalid birthday years with a placeholder (1900)
- Trim GEN and store as "Male" or "Female"
- Changing BDATE data type to date

### erp_locations

In [150]:
#Looking at CID not following the AW... format
pd.read_sql("""
SELECT CID
FROM raw.erp_locations
WHERE CID NOT LIKE 'AW-%';""", engine)

,CID


erp_locations.CID needs to match crm_customers.cst_key, but currently has an extra dash

In [121]:
#Looking at unique CNTRY values
pd.read_sql("""
SELECT DISTINCT CNTRY
FROM raw.erp_locations;""", engine)

,CNTRY
0,Australia
1,US
2,Canada
3,DE
4,United Kingdom
5,France
6,USA
7,Germany
8,None
9,


CNTRY column has inconsistent formatting

Based on this exploration, raw.erp_locations will be cleaned up by:
- Remove extra character ('-') from CID so that it matches cst_key in crm_customers
- Trim and standardize countries

### erp_product_categories

In [122]:
#Looking at unique CAT values
pd.read_sql("""
SELECT DISTINCT CAT
FROM raw.erp_product_categories;""", engine)

,CAT
0,Accessories
1,Bikes
2,Clothing
3,Components


In [123]:
#Looking at unique SUBCAT values
pd.read_sql("""
SELECT DISTINCT SUBCAT
FROM raw.erp_product_categories
ORDER BY SUBCAT ASC;""", engine)

,SUBCAT
0,Bib-Shorts
1,Bike Racks
2,Bike Stands
3,Bottles and Cages
4,Bottom Brackets
5,Brakes
6,Caps
7,Chains
8,Cleaners
9,Cranksets


In [124]:
#Looking at unique MAINTENANCE values
pd.read_sql("""
SELECT DISTINCT MAINTENANCE
FROM raw.erp_product_categories;""", engine)

,MAINTENANCE
0,Yes
1,No


No formatting issues across CAT, SUBCAT, MAINTENANCE columns

Based on this exploration, raw.erp_product_categories will be cleaned up by:
- erp_product_categories.ID: '_' to '-' so that it matches crm_products.prd_key